# Pumpkinmeter - A Movie Recommendation Service
### Source: https://www.codementor.io/spark/tutorial/building-a-recommender-with-apache-spark-python-example-app-part1

#### Create a SparkContext configured for local mode

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

#### File download
We will use the latest datasets:

Small: 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Last updated 9/2018.

Full: 27,000,000 ratings and 1,100,000 tag applications applied to 58,000 movies by 280,000 users.
Includes tag genome data with 14 million relevance scores across 1,100 tags. Last updated 9/2018.

In [2]:
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'

#### Download location(s)


In [3]:
import os
datasets_path = os.path.join('/home/jovyan', 'work')
small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')
complete_dataset_path = os.path.join(datasets_path, 'ml-latest.zip')

#### Getting file(s)

In [4]:
import urllib.request
small_f = urllib.request.urlretrieve (small_dataset_url, small_dataset_path)
complete_f = urllib.request.urlretrieve (complete_dataset_url, complete_dataset_path)

#### Extracting file(s)


In [5]:
import zipfile

with zipfile.ZipFile(small_dataset_path, "r") as z:
    z.extractall(datasets_path)
    
with zipfile.ZipFile(complete_dataset_path, "r") as z:
    z.extractall(datasets_path)

## Loading and parsing datasets
Now we are ready to read in each of the files and create an RDD consisting of parsed lines. 

Each line in the ratings dataset (ratings.csv) is formatted as: 
+ userId,movieId,rating,timestamp 

Each line in the movies (movies.csv) dataset is formatted as:
+ movieId,title,genres 

The format of these files is uniform and simple, so we can use Python split() to parse their lines once they are loaded into RDDs. Parsing the movies and ratings files yields two RDDs: 
+ For each line in the ratings dataset, we create a tuple of (UserID, MovieID, Rating). We drop the timestamp because we do not need it for this recommender.
+ For each line in the movies dataset, we create a tuple of (MovieID, Title). We drop the genres because we do not use them for this recommender.

So let's load the raw ratings data. We need to filter out the header, included in each file.

#### ratings.csv

In [6]:

small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')
small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]
# Parse
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()

print ('There are {} recommendations in the small dataset'.format(small_ratings_data.count()))
small_ratings_data.take(3)

There are 100836 recommendations in the small dataset


[(1, 1, 4.0), (1, 3, 4.0), (1, 6, 4.0)]

#### movies.csv


In [7]:

# Load the small dataset file
small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')
small_movies_raw_data = sc.textFile(small_movies_file)
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

# Parse
small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

small_movies_titles = small_movies_data.map(lambda x: (int(x[0]),x[1]))
print ('There are {} movies in the small dataset'.format(small_movies_titles.count()))
small_movies_data.take(3)


There are 9742 movies in the small dataset


[(1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
 (2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
 (3, 'Grumpier Old Men (1995)', 'Comedy|Romance')]

## Collaborative Filtering
In Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly. 

At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has the following parameters:

+ numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
+ rank is the number of latent factors in the model.
+ iterations is the number of iterations to run.
+ lambda specifies the regularization parameter in ALS.
+ implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
+ alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

#### Selecting ALS parameters using the small dataset
In order to determine the best ALS parameters, we will use the small dataset. We need first to split it into train, validation, and test datasets.


In [8]:

training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=0)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

#### Training phase

In [9]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1

for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print ('For rank {} the RMSE is {}'.format(rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank {}'.format(best_rank))

For rank 4 the RMSE is 0.908078105265682
For rank 8 the RMSE is 0.916462973348527
For rank 12 the RMSE is 0.917665030756129
The best model was trained with rank 4


In [10]:
# predictions
predictions.take(3)

[((372, 1084), 3.42419871162954),
 ((4, 1084), 3.866749726695713),
 ((402, 1084), 3.4099577968422152)]

We have the UserID, the MovieID, and the Rating, as we have in our ratings dataset. In this case the predictions third element, the rating for that movie and user, is the predicted by our ALS model.

Then we join these with our validation data (the one that includes ratings) and the result looks as follows:

In [11]:
rates_and_preds.take(3)

[((1, 457), (5.0, 4.381060760461434)),
 ((1, 1025), (5.0, 4.705295366590298)),
 ((1, 1089), (5.0, 4.979982471805129))]

To that, we apply a squared difference and the we use the mean() action to get the MSE and apply sqrt.

Finally we test the selected model.

In [12]:
model = ALS.train(training_RDD, best_rank, seed=seed, \
                           iterations=iterations, lambda_=regularization_parameter)

predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

print ('For testing data the RMSE is {}'.format(error))

For testing data the RMSE is 0.9113780946334407


## Using the complete dataset to build the final model

In order to build our recommender model, we will use the complete dataset. Therefore, we need to process it the same way we did with the small dataset.

ratings.csv for complete dataset

In [13]:
# Load the complete dataset file
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]
# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()

print ('There are {} recommendations in the complete dataset'.format(complete_ratings_data.count()))
complete_ratings_data.take(3)


There are 27753444 recommendations in the complete dataset


[(1, 307, 3.5), (1, 481, 3.5), (1, 1091, 1.5)]

Now we are ready to train the recommender model.

In [14]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=0)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, \
                           iterations=iterations, lambda_=regularization_parameter)

Now we test on our testing set.

In [15]:
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

print ('For testing data the RMSE is {}'.format(error))

For testing data the RMSE is 0.8318265262101795


We can see how we got a more accurate recommender when using a much larger dataset.

## How to make recommendations
Although we aim at building an online movie recommender, now that we know how to have our recommender model ready, we can give it a try providing some movie recommendations. This will help us coding the recommending engine later on when building the web service, and will explain how to use the model in any other circumstances.

When using collaborative filtering, getting recommendations is not as simple as predicting for the new entries using a previously generated model. Instead, we need to train again the model but including the new user preferences in order to compare them with other users in the dataset. That is, the recommender needs to be trained every time we have new user ratings (although a single model can be used by multiple users of course!). This makes the process expensive, and it is one of the reasons why scalability is a problem (and Spark a solution!). Once we have our model trained, we can reuse it to obtain top recomendations for a given user or an individual rating for a particular movie. These are less costly operations than training the model itself.


So let's first load the movies complete file for later use.

In [16]:
# Load the complete dataset file
complete_movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
print ('There are {} movies in the complete dataset'.format(complete_movies_titles.count()))
complete_movies_data.take(3)


There are 58098 movies in the complete dataset


[(1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
 (2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
 (3, 'Grumpier Old Men (1995)', 'Comedy|Romance')]

Another thing we want to do, is give recommendations of movies with a certain minimum number of ratings. For that, we need to count the number of ratings per movie.

In [17]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

## User 1
### Adding new user ratings 
Now we need to rate some movies for a new user which is myself. We will put them in a new RDD and we will use the user ID 0  that is not assigned in the MovieLens dataset. The dataset movies file is checked for ID to Tittle assignment.

In [18]:
new_user1_ID = 0

# The format of each line is (userID, movieID, rating)

# ###################################################
# Keeping the userID, but Replacing movieID, rating, title according to 10 movies you have watched in the past and my OWN ratings
# ###################################################


new_user1_ratings = [
     (0,260,4), # Star Wars: Episode IV - A New Hope (1977)
     (0,58559,4), # Dark Knight, The (2008)
     (0,48516,3), # Departed, The (2006)
     (0,593,4), # Silence of the Lambs, The (1991)
     (0,32,3), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,79132,4), # Inception (2010)
     (0,318,5), # Shawshank Redemption, The (1994)
     (0,364,2), # Lion King, The (1994)
     (0,858,5), # Godfather, The (1972)
     (0,356,5) # Forrest Gump (1994)
    ]

new_user1_ratings_RDD = sc.parallelize(new_user1_ratings)
print ('New user1 ratings: {}'.format(new_user1_ratings_RDD.take(10)))



New user1 ratings: [(0, 260, 4), (0, 58559, 4), (0, 48516, 3), (0, 593, 4), (0, 32, 3), (0, 79132, 4), (0, 318, 5), (0, 364, 2), (0, 858, 5), (0, 356, 5)]


Now we add them to the data we will use to train our recommender model. We use Spark's union() transformation for this.

In [19]:

complete_data_with_new_ratings_RDD_1 = complete_ratings_data.union(new_user1_ratings_RDD)


And finally we train the ALS model using all the parameters we selected before (when using the small dataset).

In [20]:
from time import time

t0 = time()
new_ratings_model_1 = ALS.train(complete_data_with_new_ratings_RDD_1, best_rank, seed=seed,
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print ('New model trained in {} seconds'.format(round(tt,3)))

New model trained in 140.546 seconds


It took some time. We will need to repeat that every time a user adds new ratings. Ideally we will do this in batches, and not for every single rating that comes into the system for every user.

### Getting top recommendations for User 1
Let's now get some recommendations! For that we will get an RDD with all the movies the new user 1 have not rated yet. We will them together with the model to predict ratings.

In [21]:
new_user1_ratings_ids = list(map(lambda x: x[1], new_user1_ratings)) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user1_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user1_ratings_ids).map(lambda x: (new_user1_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user1_recommendations_RDD = new_ratings_model_1.predictAll(new_user1_unrated_movies_RDD)


We have our recommendations ready. Now we can print out the 15 movies with the highest predicted ratings and join them with the movies RDD to get the titles, and ratings count in order to get movies with a minimum number of counts. First we will do the join and see what does the result looks like.

In [22]:
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user1_recommendations_rating_RDD = new_user1_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user1_recommendations_rating_title_and_count_RDD = \
    new_user1_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user1_recommendations_rating_title_and_count_RDD.take(3)

[(6216,
  ((3.7626955032043448, 'Nowhere in Africa (Nirgendwo in Afrika) (2001)'),
   717)),
 (124320, ((3.7490892448549467, 'Once a Thief (1965)'), 1)),
 (83916, ((3.090721472937144, 'Blues in the Night (1941)'), 9))]

So we need to flat this down a bit in order to have (Title, Rating, Ratings Count).

In [23]:
new_user1_recommendations_rating_title_and_count_RDD = \
    new_user1_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

### User 1 : Scenario 1
Get the highest rated recommendations for the new user 1, filtering out movies with less than 25 ratings.


In [24]:
top_movies_user1_25reviews = new_user1_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(15, key=lambda x: -x[1])

print ('TOP recommended movies for User 1 (with more than 25 reviews):\n{}'.format('\n'.join(map(str, top_movies_user1_25reviews))))


TOP recommended movies for User 1 (with more than 25 reviews):
('Planet Earth II (2016)', 4.3361926630893315, 853)
('Planet Earth (2006)', 4.312682184100417, 1384)
('Band of Brothers (2001)', 4.253593879561944, 984)
('Blue Planet II (2017)', 4.250443625017688, 349)
('Life (2009)', 4.2419448598883704, 166)
('The Blue Planet (2001)', 4.217744131351459, 421)
('Frozen Planet (2011)', 4.203898463119426, 402)
('Cosmos', 4.193298933313592, 157)
('"Things I Like', 4.188455804323436, 30)
('The Reichenbach Fall (2012)', 4.167207554271638, 48)
("Won't You Be My Neighbor? (2018)", 4.159217682024021, 83)
('The Godfather Trilogy: 1972-1990 (1992)', 4.153614009560567, 421)
('Music for One Apartment and Six Drummers (2001)', 4.145544574312158, 31)
('Over the Garden Wall (2013)', 4.1433365188486135, 377)
('The Farthest (2017)', 4.1273209637729416, 28)


### User 1 : Scenario 2
Get the highest rated recommendations for the new user, filtering out movies with less than 100 ratings.

In [25]:
top_movies_user1_100reviews = new_user1_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=100).takeOrdered(15, key=lambda x: -x[1])

print ('TOP recommended movies for User 1 (with more than 100 reviews):\n{}'.format('\n'.join(map(str, top_movies_user1_100reviews))))


TOP recommended movies for User 1 (with more than 100 reviews):
('Planet Earth II (2016)', 4.3361926630893315, 853)
('Planet Earth (2006)', 4.312682184100417, 1384)
('Band of Brothers (2001)', 4.253593879561944, 984)
('Blue Planet II (2017)', 4.250443625017688, 349)
('Life (2009)', 4.2419448598883704, 166)
('The Blue Planet (2001)', 4.217744131351459, 421)
('Frozen Planet (2011)', 4.203898463119426, 402)
('Cosmos', 4.193298933313592, 157)
('The Godfather Trilogy: 1972-1990 (1992)', 4.153614009560567, 421)
('Over the Garden Wall (2013)', 4.1433365188486135, 377)
('Alone in the Wilderness (2004)', 4.125838776352651, 343)
('"Civil War', 4.117653583820509, 431)
('Olive Kitteridge (2014)', 4.1051077804416956, 211)
('"Lives of Others', 4.097398034313056, 9670)
("Schindler's List (1993)", 4.093065821867768, 71516)


## Getting individual ratings
Another useful usecase is getting the predicted rating for a particular movie for a given user. The process is similar to the previous retreival of top recommendations but, instead of using predcitAll with every single movie the user hasn't rated yet, we will just pass the method a single entry with the movie we want to predict the rating for.

In [27]:
my_movie = sc.parallelize([(0, 920)]) # Gone with the Wind (1939)
individual_movie_rating_RDD = new_ratings_model_1.predictAll(new_user1_unrated_movies_RDD)
individual_movie_rating_RDD.take(1)


[Rating(user=0, product=116688, rating=1.0475738885029453)]

## Persisting the model
Optionally, we might want to persist the base model for later use in our on-line recommendations. Although a new model is generated everytime we have new user ratings, it might be worth it to store the current one, in order to save time when starting up the server, etc. We might also save time if we persist some of the RDDs we have generated, specially those that took longer to process. For example, the following lines save and load a ALS model.

In [28]:
from pyspark.mllib.recommendation import MatrixFactorizationModel

model_path = os.path.join('..', 'models', 'movie_lens_als')

# Save and load model
model.save(sc, model_path)
same_model = MatrixFactorizationModel.load(sc, model_path)